# 1. Import các thư viện

In [35]:
from random import random
#Thư viện xử lý và tính toán dữ liệu
import tensorflow as tf
import numpy as np
import math
import pandas as pd
#Thư viện vẽ biểu đồ
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
#Thư viện xây dựng hàm dự đoán
from keras.src.callbacks import ModelCheckpoint
from keras.src.layers import Bidirectional, BatchNormalization, Conv1D
from keras.src.saving import load_model
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

# 2. Import dữ liệu

Dữ liệu được lấy bằng cách sử dụng API của CTCP Chứng khoán SSI
Dữ liệu trả về là file CSV

In [36]:
data_vcb = pd.read_csv('Data\VCB_Historical_Data.csv')
data_fpt = pd.read_csv('Data\FPT_Historical_Data.csv')
data_mwg = pd.read_csv('Data\MWG_Historical_Data.csv')
data_plx = pd.read_csv('Data\PLX_Historical_Data.csv')
data_ssi = pd.read_csv('Data\SSI_Historical_Data.csv')
print(data_vcb)
print(data_fpt)
print(data_mwg)
print(data_plx)
print(data_ssi)

     Symbol Market TradingDate  Time   Open   High    Low  Close   Volume  \
0       VCB   HOSE  02/01/2019   NaN  34738  34995  34480  34544  1081640   
1       VCB   HOSE  03/01/2019   NaN  34802  34867  33964  34544  1071350   
2       VCB   HOSE  04/01/2019   NaN  34351  35060  33900  35060  1307310   
3       VCB   HOSE  07/01/2019   NaN  35640  35769  35124  35189  1175810   
4       VCB   HOSE  08/01/2019   NaN  35447  35575  35124  35447  1318810   
...     ...    ...         ...   ...    ...    ...    ...    ...      ...   
1318    VCB   HOSE  12/04/2024   NaN  94500  95500  93800  94600   844800   
1319    VCB   HOSE  15/04/2024   NaN  94600  96500  92000  92000  1900300   
1320    VCB   HOSE  16/04/2024   NaN  93800  93800  90500  91000  1857700   
1321    VCB   HOSE  17/04/2024   NaN  91200  92100  89500  90500  1452600   
1322    VCB   HOSE  19/04/2024   NaN  90000  90900  89700  90500  1324800   

             Value  
0     5.831230e+10  
1     5.733404e+10  
2     7.0075

# 3. Mô tả dữ liệu

Cột TradingDate có Dtype là object vì trong file csv, nó có kiểu dữ liệu là string, không phải là datetime, được cast sang object khi import vào bằng pandas

In [37]:
data_vcb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1323 entries, 0 to 1322
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Symbol       1323 non-null   object 
 1   Market       1323 non-null   object 
 2   TradingDate  1323 non-null   object 
 3   Time         0 non-null      float64
 4   Open         1323 non-null   int64  
 5   High         1323 non-null   int64  
 6   Low          1323 non-null   int64  
 7   Close        1323 non-null   int64  
 8   Volume       1323 non-null   int64  
 9   Value        1323 non-null   float64
dtypes: float64(2), int64(5), object(3)
memory usage: 103.5+ KB


In [38]:
data_fpt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1323 entries, 0 to 1322
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Symbol       1323 non-null   object 
 1   Market       1323 non-null   object 
 2   TradingDate  1323 non-null   object 
 3   Time         0 non-null      float64
 4   Open         1323 non-null   int64  
 5   High         1323 non-null   int64  
 6   Low          1323 non-null   int64  
 7   Close        1323 non-null   int64  
 8   Volume       1323 non-null   int64  
 9   Value        1323 non-null   float64
dtypes: float64(2), int64(5), object(3)
memory usage: 103.5+ KB


In [39]:
data_mwg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1323 entries, 0 to 1322
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Symbol       1323 non-null   object 
 1   Market       1323 non-null   object 
 2   TradingDate  1323 non-null   object 
 3   Time         0 non-null      float64
 4   Open         1323 non-null   int64  
 5   High         1323 non-null   int64  
 6   Low          1323 non-null   int64  
 7   Close        1323 non-null   int64  
 8   Volume       1323 non-null   int64  
 9   Value        1323 non-null   float64
dtypes: float64(2), int64(5), object(3)
memory usage: 103.5+ KB


In [40]:
data_plx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1323 entries, 0 to 1322
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Symbol       1323 non-null   object 
 1   Market       1323 non-null   object 
 2   TradingDate  1323 non-null   object 
 3   Time         0 non-null      float64
 4   Open         1323 non-null   int64  
 5   High         1323 non-null   int64  
 6   Low          1323 non-null   int64  
 7   Close        1323 non-null   int64  
 8   Volume       1323 non-null   int64  
 9   Value        1323 non-null   float64
dtypes: float64(2), int64(5), object(3)
memory usage: 103.5+ KB


In [41]:
data_ssi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1323 entries, 0 to 1322
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Symbol       1323 non-null   object 
 1   Market       1323 non-null   object 
 2   TradingDate  1323 non-null   object 
 3   Time         0 non-null      float64
 4   Open         1323 non-null   int64  
 5   High         1323 non-null   int64  
 6   Low          1323 non-null   int64  
 7   Close        1323 non-null   int64  
 8   Volume       1323 non-null   int64  
 9   Value        1323 non-null   float64
dtypes: float64(2), int64(5), object(3)
memory usage: 103.5+ KB


# 4. Đưa dữ liệu vào Data Frame

In [59]:
df_vcb = pd.DataFrame(data_vcb,columns=['TradingDate','Open','High','Low','Close','Volume'])
df_fpt = pd.DataFrame(data_fpt,columns=['TradingDate','Open','High','Low','Close','Volume'])
df_mwg = pd.DataFrame(data_mwg,columns=['TradingDate','Open','High','Low','Close','Volume'])
df_plx = pd.DataFrame(data_plx,columns=['TradingDate','Open','High','Low','Close','Volume'])
df_ssi = pd.DataFrame(data_ssi,columns=['TradingDate','Open','High','Low','Close','Volume'])

## Định dạng lại cột TradingDate thành YYYY-mm-dd

In [60]:
df_vcb['TradingDate'] = pd.to_datetime(df_vcb['TradingDate'], format='%d/%m/%Y')
df_vcb.set_index('TradingDate', inplace=True)

df_fpt['TradingDate'] = pd.to_datetime(df_fpt['TradingDate'], format='%d/%m/%Y')
df_fpt.set_index('TradingDate', inplace=True)

df_mwg['TradingDate'] = pd.to_datetime(df_mwg['TradingDate'], format='%d/%m/%Y')
df_mwg.set_index('TradingDate', inplace=True)

df_plx['TradingDate'] = pd.to_datetime(df_plx['TradingDate'], format='%d/%m/%Y')
df_plx.set_index('TradingDate', inplace=True)

df_ssi['TradingDate'] = pd.to_datetime(df_ssi['TradingDate'], format='%d/%m/%Y')
df_ssi.set_index('TradingDate', inplace=True)

## Dữ liệu của Data Frame

In [61]:
df_vcb

,Open,High,Low,Close,Volume
TradingDate,,,,,
2019-01-02,34738,34995,34480,34544,1081640
2019-01-03,34802,34867,33964,34544,1071350
2019-01-04,34351,35060,33900,35060,1307310
2019-01-07,35640,35769,35124,35189,1175810
2019-01-08,35447,35575,35124,35447,1318810
...,...,...,...,...,...
2024-04-12,94500,95500,93800,94600,844800
2024-04-15,94600,96500,92000,92000,1900300
2024-04-16,93800,93800,90500,91000,1857700


In [62]:
df_fpt

,Open,High,Low,Close,Volume
TradingDate,,,,,
2019-01-02,18036,18187,17756,17777,459120
2019-01-03,17885,17885,17562,17605,716600
2019-01-04,17518,17885,17475,17734,318570
2019-01-07,18036,18166,17950,18036,408470
2019-01-08,18036,18122,17993,18122,295660
...,...,...,...,...,...
2024-04-12,115900,115900,114300,115500,1367300
2024-04-15,115500,115600,111500,111600,3701900
2024-04-16,112000,113200,111500,113000,3642700


In [63]:
df_mwg

,Open,High,Low,Close,Volume
TradingDate,,,,,
2019-01-02,27583,27583,27107,27107,351730
2019-01-03,27107,27170,26314,26314,703060
2019-01-04,26314,26536,25997,26505,432170
2019-01-07,26631,27107,26568,26948,398000
2019-01-08,26948,26948,26631,26631,172180
...,...,...,...,...,...
2024-04-12,52300,52600,51600,52000,6665600
2024-04-15,51700,52000,49500,49900,16505200
2024-04-16,49500,50000,48700,49900,11616300


In [64]:
df_ssi

,Open,High,Low,Close,Volume
TradingDate,,,,,
2019-01-02,12102,12193,11898,12011,1216350
2019-01-03,12011,12011,11671,11785,1672220
2019-01-04,11785,11875,11535,11785,1232910
2019-01-07,12147,12147,11807,11807,735500
2019-01-08,11853,11853,11626,11649,1306960
...,...,...,...,...,...
2024-04-12,37350,37800,37050,37550,19498500
2024-04-15,37300,37650,34950,34950,29936900
2024-04-16,35100,35300,34100,35200,32795000


In [65]:
df_plx

,Open,High,Low,Close,Volume
TradingDate,,,,,
2019-01-02,44638,45878,44638,44969,380020
2019-01-03,44969,45465,43398,44307,450490
2019-01-04,43811,45630,43811,45382,302070
2019-01-07,45878,46043,45547,45630,187920
2019-01-08,45878,45878,44059,44721,352050
...,...,...,...,...,...
2024-04-12,36500,36950,36350,36850,445900
2024-04-15,36900,37200,34800,36000,927300
2024-04-16,36000,36300,35150,35950,786100


Mô tả dữ liệu Data Frame

In [66]:
df_vcb.describe()

,Open,High,Low,Close,Volume
count,1323.000000,1323.000000,1323.000000,1323.000000,1.323000e+03
mean,64832.292517,65560.201814,64110.126228,64876.920635,1.142606e+06
std,14444.890834,14537.553736,14307.695174,14429.454007,6.259926e+05
min,34351.000000,34867.000000,33900.000000,34544.000000,1.414500e+05
25%,54699.500000,55293.500000,54269.500000,54732.000000,7.156000e+05
50%,64240.000000,64896.000000,63519.000000,64240.000000,1.020600e+06
75%,73663.000000,74696.000000,72864.500000,73917.000000,1.427750e+06
max,98500.000000,100500.000000,97300.000000,97400.000000,6.274200e+06


In [67]:
df_fpt.describe()

,Open,High,Low,Close,Volume
count,1323.000000,1323.000000,1323.000000,1323.000000,1.323000e+03
mean,54285.768707,54888.482237,53724.286470,54313.755858,1.808710e+06
std,25957.884113,26234.620646,25677.811185,25959.202387,1.131201e+06
min,17518.000000,17885.000000,17475.000000,17605.000000,1.414800e+05
25%,28040.000000,28237.000000,27793.000000,27991.000000,1.013075e+06
50%,61796.000000,62873.000000,61033.000000,62023.000000,1.546300e+06
75%,70533.500000,71052.000000,69627.500000,70434.000000,2.296050e+06
max,118500.000000,118500.000000,116000.000000,117100.000000,8.399400e+06


In [68]:
df_mwg.describe()

,Open,High,Low,Close,Volume
count,1323.000000,1323.000000,1323.000000,1323.000000,1.323000e+03
mean,44670.708995,45281.724112,44047.161754,44695.532880,2.611731e+06
std,13922.695079,14145.491090,13653.633879,13918.595757,3.296921e+06
min,18930.000000,20059.000000,18156.000000,18995.000000,1.415900e+05
25%,35136.000000,35426.000000,34665.000000,35135.500000,7.360000e+05
50%,42146.000000,42600.000000,41595.000000,42195.000000,1.266900e+06
75%,53900.000000,54500.000000,52904.500000,53800.000000,2.808200e+06
max,79217.000000,80103.000000,78282.000000,78823.000000,2.238030e+07


In [69]:
df_plx.describe()

,Open,High,Low,Close,Volume
count,1323.000000,1323.000000,1323.000000,1323.00000,1.323000e+03
mean,43994.497354,44500.665911,43459.114890,43928.97808,1.180151e+06
std,7576.094483,7603.530891,7519.489611,7553.41143,1.016567e+06
min,22553.000000,25305.000000,22553.000000,24224.00000,1.986000e+04
25%,37337.000000,37716.000000,36851.000000,37240.50000,5.098600e+05
50%,45246.000000,45707.000000,44645.000000,45153.00000,9.232000e+05
75%,50731.000000,51248.000000,50230.500000,50682.50000,1.494350e+06
max,59485.000000,62124.000000,58825.000000,59767.00000,7.387900e+06


In [70]:
df_ssi.describe()

,Open,High,Low,Close,Volume
count,1323.000000,1323.000000,1323.000000,1323.000000,1.323000e+03
mean,21178.103553,21536.174603,20819.984883,21160.665911,1.229921e+07
std,11004.353478,11192.652376,10808.016515,10981.730983,9.980044e+06
min,5492.000000,5634.000000,5256.000000,5445.000000,3.946400e+05
25%,10787.000000,10895.500000,10593.500000,10734.500000,3.422960e+06
50%,19434.000000,19723.000000,19074.000000,19454.000000,1.115630e+07
75%,31525.000000,32047.500000,30658.000000,31357.000000,1.802685e+07
max,49368.000000,50151.000000,47804.000000,48586.000000,6.955910e+07


# 5. Trực quan hoá dữ liệu

## 5.1 Hàm vẽ biểu đồ nến giá cổ phiếu

In [71]:
def plot_candlestick(stock_df_name, name = '', fig_size=(1000, 700)):
    stock_data = stock_df_name.copy()

    fig = go.Figure(data=[go.Candlestick(x=stock_data.index,
                                         close=stock_data['Close'],
                                         open=stock_data['Open'],
                                         high=stock_data['High'],
                                         low=stock_data['Low'],
                                         name="Candlesticks",
                                         increasing_line_color='green',
                                         decreasing_line_color='red', line=dict(width=1)
                                         )])
    fig.update_xaxes(
        rangeslider_visible=True,
        rangebreaks=[
            # NOTE: Below values are bound (not single values), ie. hide x to y
            dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
        ]
    )
    
    fig.update_layout(title=f"Biểu đồ nến giá cổ phiếu {name}",
                      xaxis_title="Ngày", yaxis_title="Giá",
                      width=fig_size[0], height=fig_size[1],
                      xaxis=dict(
                          rangeselector=dict(
                              buttons=list([
                                dict(count=1, label="1m", step="month", stepmode="backward"),
                                  dict(count=3, label="3m", step="month", stepmode="backward"),
                                  dict(count=6, label="6m", step="month", stepmode="backward"),
                                  dict(count=1, label="YTD", step="year", stepmode="todate"),
                                  dict(count=1, label="1y", step="year", stepmode="backward"),
                                  dict(count=3, label="3y", step="year", stepmode="backward"),
                                  dict(count=5, label="5y", step="year", stepmode="backward"),
                                  dict(step="all")]),
                              bgcolor='pink',
                              font=dict(color='black'),
                              activecolor='lightgreen'))
                      )
    fig.show()

### Vẽ biểu đồ 

In [72]:
symbol_dict = {
    'VCB' : df_vcb,
    'FPT' : df_fpt,
    'MWG' : df_mwg,
    'PLX' : df_plx,
    'SSI' : df_ssi
}

for name, df in symbol_dict.items():
    plot_candlestick(df, name)

## 5.2 Biểu đồ biến động giá và khối lượng giao dịch 

In [73]:
# Creating subplot
def plot_perchange_volume(stock_df, name = ''):
    fig = make_subplots(rows=2, cols=2, column_widths=[0.7, 0.3],
                        vertical_spacing=0.1, horizontal_spacing=0.05,
                        subplot_titles=(f"{name} - Biến động giá theo thời gian", f"{name} Phần trăm thay đổi - Histogram",
                                        f"{name} - Khối lượng giao dịch", f"{name} Khối lượng giao dịch - Histogram",))
    # Percent Change Plot
    percent_change = stock_df['Close'].pct_change() * 100
    fig.add_trace(go.Scatter(x=df.index, y=percent_change, name='Phần trăm thay đổi', marker_color='darkorchid'), 
                  row=1, col=1)
    fig.add_trace(go.Histogram(x=percent_change, nbinsx=50, name='Phần trăm thay đổi', marker_color='darkorchid'),  
                  row=1, col=2)
    fig.add_annotation(text=f"Mean: {percent_change.mean():.2f}%<br>Std Dev: {percent_change.std():.2f}%",
                       xref='x2', yref='y2', x=percent_change.mean(), y=5, showarrow=True)
    # Volume Plot
    fig.add_trace(go.Scatter(x=df.index, y=df['Volume'], name='Khối lượng giao dịch', marker_color='darkcyan'), 
                  row=2, col=1)
    fig.add_trace(go.Histogram(x=df['Volume'], nbinsx=50, name='Khối lượng giao dịch', marker_color='darkcyan'),  
                  row=2, col=2)
    fig.add_annotation(text=f"Mean: {df['Volume'].mean():.2f}<br>Std Dev: {df['Volume'].std():.2f}",
                       xref='x4', yref='y4', x=df['Volume'].mean(), y=5, showarrow=True)

    fig.update_layout(height=700, width=1100)
    fig.show()

### Vẽ biểu đồ

In [74]:
for name, df in symbol_dict.items():
    plot_perchange_volume(df, name)

# 6. Tiền xử lý dữ liệu

## 6.1 Hàm chuẩn hoá dữ liệu

In [81]:
def data_mapping(data, n_steps_in, n_steps_out):
    data_scaler = data.copy()
    scaler = MinMaxScaler(feature_range=(0, 1))
    normalized_data = scaler.fit_transform(data_scaler)
    num_samples = len(normalized_data) - n_steps_in - n_steps_out + 1

    X = np.empty((num_samples, n_steps_in, 1))
    Y = np.empty((num_samples, n_steps_out, 1))

    for i in range(num_samples):
        X[i, :, :] = normalized_data[i:i + n_steps_in, :]
        Y[i, :, :] = normalized_data[i + n_steps_in:i + n_steps_in + n_steps_out, :]

    return X, Y

## 6.2 Hàm chia tập dữ liệu

In [82]:
def train_test_split(X, Y, test_percent=0.2, random_state=42):
    if random_state is not None:
        np.random.seed(random_state)

    test_size = int(len(X) * test_percent)

    indices = np.arange(len(X))
    np.random.shuffle(indices)

    train_indices = indices[:-test_size]
    test_indices = indices[-test_size:]

    X_train, Y_train = X[train_indices], Y[train_indices]
    X_test, Y_test = X[test_indices], Y[test_indices]

    return X_train, X_test, Y_train, Y_test

# 7. Dự đoán giá cổ phiếu

In [83]:
def multi_step_LSTM_forecast(X_train, y_train, X_test, y_test, n_steps_in, n_steps_out, data: np.array):
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    model = Sequential()

    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_steps_in, 1)))
    model.add(Dropout(0.2))

    model.add(LSTM(units=32, return_sequences=True, input_shape=(n_steps_in, 1)))
    model.add(Dropout(0.2))

    model.add(Bidirectional(LSTM(units=32, return_sequences=True)))
    model.add(BatchNormalization())

    model.add(LSTM(units=32, return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(units=n_steps_out))

    model.compile(loss='mse',optimizer='adam')
    model.summary()

    save_model = "save_model.keras"
    best_model = ModelCheckpoint(save_model, monitor='loss', verbose=1, save_best_only=True, mode='auto')

    model.fit(X_train, y_train.reshape(-1, n_steps_out),
                 batch_size=64,
                 epochs=50,
                 validation_data=(X_test, y_test.reshape(-1, n_steps_out)),
                 callbacks=[best_model],
                 validation_split=0.1,
                 shuffle=False,
                 verbose=1)

    # Normalizing latest data
    last_steps_normalized = scaler.fit_transform(data)
    last_steps_normalized = last_steps_normalized.reshape(1, n_steps_in, 1)

    # Predicting using model
    forecast = model.predict(last_steps_normalized)
    forecast = forecast.reshape(n_steps_out, 1)

    # Inverse transforming to original scale
    forecast = scaler.inverse_transform(forecast)
    forecast = pd.DataFrame(forecast,columns=['Close'])
    
    evaluate_model(model, scaler, X_train, X_test, y_train, y_test, n_steps_out)
    
    return forecast

## 7.1 Hàm đánh giá mô hình

In [84]:
def evaluate_model(model, scaler, x_train, x_test, y_train, y_test, n_steps_out):
    # Dự đoán trên tập huấn luyện và tập kiểm tra
    train_predict = model.predict(x_train)
    test_predict = model.predict(x_test)

    # Chuyển đổi dự đoán về dạng ban đầu trước khi chuẩn hóa
    train_predict = scaler.inverse_transform(train_predict)
    test_predict = scaler.inverse_transform(test_predict)

    # Tính toán độ lỗi
    train_predict_2d = train_predict.reshape(-1, n_steps_out)
    test_predict_2d = test_predict.reshape(-1, n_steps_out)
    y_train_2d = y_train.reshape(-1, n_steps_out)
    y_test_2d = y_test.reshape(-1, n_steps_out)

    # Tính toán độ lỗi
    mse_train = mean_squared_error(y_train_2d, train_predict_2d)
    mae_train = mean_absolute_error(y_train_2d, train_predict_2d)
    mse_test = mean_squared_error(y_test_2d, test_predict_2d)
    mae_test = mean_absolute_error(y_test_2d, test_predict_2d)

    print("Training MSE:", mse_train)
    print("Training MAE:", mae_train)
    print("Testing MSE:", mse_test)
    print("Testing MAE:", mae_test)

## 7.2 Hàm vẽ biểu đồ giá dự đoán

In [85]:
def plot_forecast_close(previous_data, forecast, test_data = None, title_add=''):
    fig = go.Figure()
    fig.add_trace(
            go.Scatter(x=previous_data.index, y=previous_data['Close'], mode='lines', name='Actual',
                       line=dict(color='blue')))
    fig.add_trace(
            go.Scatter(x=forecast.index, y=forecast['Close'],
                           mode='lines', name='Forecast Close', line=dict(color='red')))
    
    fig.show()
    

## 7.3 Hàm tổng hợp

In [103]:
def main_func(df, name):
    n_steps_in = 300  # 1 năm
    n_steps_out = 30  # dự đoán cho 1 tháng

    close_data = df['Close'].values.reshape(-1, 1)
    
    X, Y = data_mapping(close_data, n_steps_in, n_steps_out)
    X_train, X_test, y_train, y_test = train_test_split(X, Y)
    
    print(f'X_train shape: {X_train.shape}\ny_train shape: {y_train.shape}')  # (time mapes, time steps, columns)
    print(f'\nX_test shape: {X_test.shape}\ny_test shape: {y_test.shape}')

    data_to_test = pd.DataFrame(df, columns=['Close']).iloc[-n_steps_in:]
    print(f"Test data :{data_to_test.info}")

    forecast = multi_step_LSTM_forecast(X_train, y_train, X_test, y_test, n_steps_in, n_steps_out, data_to_test.values.reshape(-1, 1))
   
    forecast.index = pd.date_range(start="04/19/2024", periods=n_steps_out)
    print(f'Forecast shape: {forecast.shape}')
    print(forecast)

    plot_forecast_close(data_to_test, forecast, title_add='- '+ name + ' Stock')

## 7.4 Chạy mô hình

In [102]:
main_func(df_vcb, 'VCB')

X_train shape: (796, 300, 1)
y_train shape: (796, 30, 1)

X_test shape: (198, 300, 1)
y_test shape: (198, 30, 1)
Test data :<bound method DataFrame.info of              Close
TradingDate       
2023-02-09   78743
2023-02-10   80013
2023-02-13   79166
2023-02-14   78320
2023-02-15   78743
...            ...
2024-04-12   94600
2024-04-15   92000
2024-04-16   91000
2024-04-17   90500
2024-04-19   90500

[300 rows x 1 columns]>


D:\Anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

D:\Anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_8 (Conv1D)               │ (None, 298, 64)        │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 298, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_24 (LSTM)                  │ (None, 298, 32)        │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 298, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_8 (Bidirectional) │ (None, 298, 64)        │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 298, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_26 (LSTM)                  │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 30)             │           990 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,974 (167.87 KB)

 Trainable params: 42,846 (167.37 KB)

 Non-trainable params: 128 (512.00 B)

Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - loss: 0.3245
Epoch 1: loss improved from inf to 0.26685, saving model to save_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 254ms/step - loss: 0.3204 - val_loss: 0.2673
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - loss: 0.1447
Epoch 2: loss improved from 0.26685 to 0.12935, saving model to save_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 197ms/step - loss: 0.1436 - val_loss: 0.2115
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - loss: 0.0837
Epoch 3: loss improved from 0.12935 to 0.07678, saving model to save_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 202ms/step - loss: 0.0832 - val_loss: 0.1849
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step - loss: 0.0600
Epoch 4: loss improved from 0.07678 to 0.05558, saving model to save_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 212ms/step - loss: 0.0597 - val_loss: 0.1794
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step - loss: 0.0425
Epoch 5: loss improved from 0.05558 to 0.042

In [88]:
main_func(df_fpt, 'FPT')

X_train shape: (796, 300, 1)
y_train shape: (796, 30, 1)

X_test shape: (198, 300, 1)
y_test shape: (198, 30, 1)
Test data :<bound method DataFrame.info of               Close
TradingDate        
2023-02-09    68565
2023-02-10    68565
2023-02-13    68565
2023-02-14    68650
2023-02-15    69330
...             ...
2024-04-12   115500
2024-04-15   111600
2024-04-16   113000
2024-04-17   111800
2024-04-19   109000

[300 rows x 1 columns]>


D:\Anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

D:\Anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 298, 64)        │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 298, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 298, 32)        │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 298, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 298, 64)        │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 298, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │           990 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,974 (167.87 KB)

 Trainable params: 42,846 (167.37 KB)

 Non-trainable params: 128 (512.00 B)

Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step - loss: 0.2493
Epoch 1: loss improved from inf to 0.19959, saving model to save_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 322ms/step - loss: 0.2458 - val_loss: 0.1921
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - loss: 0.1058
Epoch 2: loss improved from 0.19959 to 0.09384, saving model to save_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 254ms/step - loss: 0.1049 - val_loss: 0.1692
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step - loss: 0.0607
Epoch 3: loss improved from 0.09384 to 0.05399, saving model to save_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 255ms/step - loss: 0.0602 - val_loss: 0.1597
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - loss: 0.0415
Epoch 4: loss improved from 0.05399 to 0.03907, saving model to save_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 251ms/step - loss: 0.0413 - val_loss: 0.1571
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - loss: 0.0306
Epoch 5: loss improved from 0.03907 to 0.031

In [89]:
main_func(df_mwg, 'MWG')

X_train shape: (796, 300, 1)
y_train shape: (796, 30, 1)

X_test shape: (198, 300, 1)
y_test shape: (198, 30, 1)
Test data :<bound method DataFrame.info of              Close
TradingDate       
2023-02-09   43582
2023-02-10   41700
2023-02-13   42096
2023-02-14   42096
2023-02-15   42344
...            ...
2024-04-12   52000
2024-04-15   49900
2024-04-16   49900
2024-04-17   49400
2024-04-19   48200

[300 rows x 1 columns]>


D:\Anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

D:\Anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 298, 64)        │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 298, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 298, 32)        │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 298, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 298, 64)        │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 298, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 30)             │           990 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,974 (167.87 KB)

 Trainable params: 42,846 (167.37 KB)

 Non-trainable params: 128 (512.00 B)

Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - loss: 0.2801
Epoch 1: loss improved from inf to 0.22552, saving model to save_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 309ms/step - loss: 0.2762 - val_loss: 0.2234
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step - loss: 0.1197
Epoch 2: loss improved from 0.22552 to 0.09764, saving model to save_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 242ms/step - loss: 0.1182 - val_loss: 0.1766
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step - loss: 0.0594
Epoch 3: loss improved from 0.09764 to 0.05498, saving model to save_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 250ms/step - loss: 0.0591 - val_loss: 0.1622
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step - loss: 0.0441
Epoch 4: loss improved from 0.05498 to 0.04264, saving model to save_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 243ms/step - loss: 0.0440 - val_loss: 0.1587
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - loss: 0.0355
Epoch 5: loss improved from 0.04264 to 0.035

In [90]:
main_func(df_plx, 'PLX')

X_train shape: (796, 300, 1)
y_train shape: (796, 30, 1)

X_test shape: (198, 300, 1)
y_test shape: (198, 30, 1)
Test data :<bound method DataFrame.info of              Close
TradingDate       
2023-02-09   37244
2023-02-10   37637
2023-02-13   37244
2023-02-14   37343
2023-02-15   38080
...            ...
2024-04-12   36850
2024-04-15   36000
2024-04-16   35950
2024-04-17   35300
2024-04-19   34650

[300 rows x 1 columns]>


D:\Anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

D:\Anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 298, 64)        │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 298, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 298, 32)        │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 298, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 298, 64)        │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 298, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 30)             │           990 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,974 (167.87 KB)

 Trainable params: 42,846 (167.37 KB)

 Non-trainable params: 128 (512.00 B)

Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - loss: 0.2532
Epoch 1: loss improved from inf to 0.20340, saving model to save_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 302ms/step - loss: 0.2496 - val_loss: 0.2869
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step - loss: 0.1025
Epoch 2: loss improved from 0.20340 to 0.09189, saving model to save_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 251ms/step - loss: 0.1018 - val_loss: 0.2434
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step - loss: 0.0635
Epoch 3: loss improved from 0.09189 to 0.05881, saving model to save_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 240ms/step - loss: 0.0632 - val_loss: 0.2255
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step - loss: 0.0469
Epoch 4: loss improved from 0.05881 to 0.04576, saving model to save_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 240ms/step - loss: 0.0468 - val_loss: 0.2243
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step - loss: 0.0391
Epoch 5: loss improved from 0.04576 to 0.039

In [91]:
main_func(df_ssi, 'SSI')

X_train shape: (796, 300, 1)
y_train shape: (796, 30, 1)

X_test shape: (198, 300, 1)
y_test shape: (198, 30, 1)
Test data :<bound method DataFrame.info of              Close
TradingDate       
2023-02-09   18617
2023-02-10   18280
2023-02-13   17799
2023-02-14   18184
2023-02-15   18376
...            ...
2024-04-12   37550
2024-04-15   34950
2024-04-16   35200
2024-04-17   34400
2024-04-19   33200

[300 rows x 1 columns]>


D:\Anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

D:\Anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_4 (Conv1D)               │ (None, 298, 64)        │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 298, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_12 (LSTM)                  │ (None, 298, 32)        │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 298, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ (None, 298, 64)        │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 298, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 30)             │           990 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,974 (167.87 KB)

 Trainable params: 42,846 (167.37 KB)

 Non-trainable params: 128 (512.00 B)

Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - loss: 0.2356
Epoch 1: loss improved from inf to 0.19076, saving model to save_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 312ms/step - loss: 0.2324 - val_loss: 0.1979
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step - loss: 0.1030
Epoch 2: loss improved from 0.19076 to 0.08980, saving model to save_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 247ms/step - loss: 0.1021 - val_loss: 0.1708
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - loss: 0.0573
Epoch 3: loss improved from 0.08980 to 0.05379, saving model to save_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 247ms/step - loss: 0.0571 - val_loss: 0.1615
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - loss: 0.0412
Epoch 4: loss improved from 0.05379 to 0.03999, saving model to save_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 248ms/step - loss: 0.0411 - val_loss: 0.1574
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step - loss: 0.0358
Epoch 5: loss improved from 0.03999 to 0.034